<a href="https://colab.research.google.com/github/boshradarabi/Message_Classifier/blob/main/Task2_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**1. Connect to Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##**2. Build Dataset**

In [2]:
import pandas as pd
import numpy as np
import re

class DatasetBuilder:
    def __init__(self):
        self.urgent_keywords = [
            'urgent', 'asap', 'emergency', 'immediately', 'critical','important', 'deadline', 'alert',
            'warning', 'attention', 'breaking', 'now', 'quick', 'hurry', 'rush'
        ]

        self.negative_urgent_patterns = [
            r'\bnot urgent\b',
            r"\bit'?s not urgent\b",
            r"don't worry\b",
            r"\bno emergency\b",
            r"\bnot critical\b"
        ]

        self.real_urgent = [
            "URGENT: Server down! Please check immediately!",
            "EMERGENCY: Critical security breach detected!",
            "ASAP: Need your approval on the contract today!",
            "IMPORTANT: Deadline extended to 5 PM today",
            "CRITICAL: Payment failed, account will be suspended",
            "ATTENTION: Meeting rescheduled to NOW",
            "WARNING: System will shut down in 10 minutes",
            "Call me immediately, it's urgent",
            "ALERT: Suspicious activity on your account",
            "Urgent: Boss wants to see you right now",
            "WARNING: Hard drive failure predicted",
            "CRITICAL: SMS service offline",
            "Emergency: Fire alarm triggered in hallway",
            "ATTENTION: VPN connection unstable",
            "Important: Last chance to submit request",
            "Urgent: Need replacement part today",
            "ASAP open Teams the meeting started",
            "WARNING: Multiple login attempts blocked",
            "Emergency: Elevator stuck with people inside",
            "Critical: Network latency extremely high",
            "URGENT: Answer your email immediately",
            "ALERT: Suspicious transaction flagged",
            "ASAP bring the documents with you",
            "Important: Customer waiting at front desk"
        ]


    def create_urgent_messages(self, df, n_samples=500):
        urgent_messages = []
        ham_messages = df[df['label'] == 'ham']['message'].values

        # strategy 1: urgent keywords with ham
        for msg in ham_messages:
            if any(kw in msg.lower() for kw in self.urgent_keywords):
                urgent_messages.append(msg)
                if len(urgent_messages) >= n_samples // 3:
                    break

        # strategy 2: Augmentation
        prefixes = ['URGENT: ', 'EMERGENCY: ', 'ASAP: ', 'IMPORTANT: ', 'CRITICAL: ', 'ATTENTION: ', 'WARNING: ', 'ALERT: ']

        np.random.seed(42)
        random_hams = np.random.choice(ham_messages, size=n_samples // 3, replace=False)
        for msg in random_hams:
            prefix = np.random.choice(prefixes)
            urgent_messages.append(prefix + msg)

        # strategy 3: custom messages
        remaining = n_samples - len(urgent_messages)
        urgent_messages.extend(self.real_urgent * (remaining // len(self.real_urgent) + 1))
        urgent_messages = urgent_messages[:n_samples]

        return pd.DataFrame({'label': ['urgent'] * len(urgent_messages), 'message': urgent_messages})


    def load_data(self, csv_path):
        df = pd.read_csv(csv_path, encoding='latin-1')
        df = df[['v1', 'v2']]
        df.columns = ['label', 'message']

        urgent_df = self.create_urgent_messages(df, n_samples=500)
        df = pd.concat([df, urgent_df], ignore_index=True)
        df = df.sample(frac=1, random_state=42).reset_index(drop=True)

        df['label'] = df['label'].replace({'ham': 'normal'})

        stats = {
            'total': len(df),
            'urgent': len(df[df['label'] == 'urgent']),
            'normal': len(df[df['label'] == 'normal']),
            'spam': len(df[df['label'] == 'spam'])
        }

        print("══════════════════════════════════════════════════════════════════════════════════════════")
        print(f"\n📊 Dataset Information: {stats}")

        return df, stats

##**3. Message Classifier**

**3.1. Import Dependencies:**

In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import re
import pickle
import warnings
warnings.filterwarnings('ignore')
import difflib
from sklearn.neural_network import MLPClassifier

**3.2. preprocess and train model:**

In [13]:
class MessageClassifier:
    def __init__(self):
        self.vectorizer = TfidfVectorizer(
            max_features=2000,
            ngram_range=(1, 2),
            min_df=2,
            max_df=0.8,
            strip_accents='unicode',
            lowercase=True,
            stop_words='english'
        )

        self.models = {
            'naive_bayes': MultinomialNB(alpha=0.1),
            'logistic': LogisticRegression(max_iter=1000, C=1.0, random_state=42),
            'random_forest': RandomForestClassifier(n_estimators=100, max_depth=20, random_state=42),
            'neural_network': MLPClassifier(
                hidden_layer_sizes=(100),
                activation='relu',
                solver='adam',
                max_iter=30,
                random_state=42
            )
        }

        self.best_model = None
        self.best_model_name = None

    def preprocess_text(self, text):
        text = str(text).lower()
        text = re.sub(r'http\S+|www\S+', 'urllink', text)
        text = re.sub(r'\S+@\S+', 'emailaddr', text)
        text = re.sub(r'\d{10}|\d{3}[-.\s]?\d{3}[-.\s]?\d{4}', 'phonenumber', text)
        text = re.sub(r'[^\w\s]', ' ', text)
        text = re.sub(r'\s+', ' ', text).strip()
        return text

    def is_similar(self, word, keywords, threshold=0.85):
        # returns True, for high similarity to keywords
        for kw in keywords:
            ratio = difflib.SequenceMatcher(None, word, kw).ratio()
            if ratio >= threshold:
                return True
        return False

    def rule_based_check(self, message):
        msg = message.lower()
        tokens = msg.split()

        negative_keywords = [
            "not urgent", "not important", "not critical",
            "don't worry", "no emergency"
        ]

        positive_keywords = [
            "urgent", "asap", "important", "critical",
            "immediately", "attention", "warning", "alert", "emergency"
        ]

        # 1) check negative keywords
        for neg in negative_keywords:
            if neg in msg:
                return "normal"

        # 2) Check positive cases with typo detection
        for token in tokens:
            if self.is_similar(token, positive_keywords):
                return "urgent"

        return None

    def train(self, df):
        df['processed'] = df['message'].apply(self.preprocess_text)

        X = self.vectorizer.fit_transform(df['processed'])
        y = df['label']

        # split dataset
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y
        )

        print(f"📊 Train: {X_train.shape[0]:,}, Test: {X_test.shape[0]:,}")
        print("\n══════════════════════════════════════════════════════════════════════════════════════════\n")

        # train models
        results = {}
        print("🎯 Training Models:")

        for name, model in self.models.items():
            print(f"\n▸ {name.title()}: ")

            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            cv_scores = cross_val_score(model, X_train, y_train, cv=5)

            results[name] = {
                'model': model,
                'accuracy': acc,
                'cv_mean': cv_scores.mean(),
                'cv_std': cv_scores.std(),
                'predictions': y_pred,
                'y_test': y_test
            }

            print(f"✓ Accuracy: {acc*100:.2f}% | CV: {cv_scores.mean()*100:.2f}% (±{cv_scores.std()*100:.2f}%)")
        print("\n══════════════════════════════════════════════════════════════════════════════════════════\n")

        # choose the best
        best_name = max(results, key=lambda x: results[x]['accuracy'])
        self.best_model = results[best_name]['model']
        self.best_model_name = best_name

        print(f"🏆 Best Model: {best_name.replace('_', ' ').title()}")
        print(f"   • Accuracy: {results[best_name]['accuracy']*100:.2f}%")
        print(f"   • CV Score: {results[best_name]['cv_mean']*100:.2f}%")
        print("\n══════════════════════════════════════════════════════════════════════════════════════════\n")

        return results, X_test, y_test, self.best_model_name


    def predict(self, message):
        rule_label = self.rule_based_check(message)
        if rule_label is not None:
            return rule_label, 1.0, {rule_label: 1.0}

        # predict a message
        processed = self.preprocess_text(message)
        X = self.vectorizer.transform([processed])
        prediction = self.best_model.predict(X)[0]

        if hasattr(self.best_model, 'predict_proba'):
            proba = self.best_model.predict_proba(X)[0]
            confidence = max(proba)
            classes = self.best_model.classes_
            proba_dict = {classes[i]: proba[i] for i in range(len(classes))}
        else:
            confidence = 1.0
            proba_dict = {prediction: 1.0}

        return prediction, confidence, proba_dict

    def predict_batch(self, messages):
        # predict batch
        results = []
        for msg in messages:
            label, conf, proba = self.predict(msg)
            results.append({
                'message': msg[:70] + '...' if len(msg) > 70 else msg,
                'predicted': label,
                'confidence': f"{conf*100:.1f}%",
                'prob_normal': f"{proba.get('normal', 0)*100:.1f}%",
                'prob_spam': f"{proba.get('spam', 0)*100:.1f}%",
                'prob_urgent': f"{proba.get('urgent', 0)*100:.1f}%"
            })
        return pd.DataFrame(results)

**3.3. classification report & confution matrix method**

In [5]:
def show_detailed_report(results, X_test, y_test, best_model_name):
    best_name = best_model_name
    y_pred = results[best_name]['predictions']

    print(f"📈 classification report ({best_name.replace('_', ' ').title()})")

    print("\n" + classification_report(y_test, y_pred,
                                        target_names=['normal', 'spam', 'urgent'],
                                        digits=3))

    print("\n══════════════════════════════════════════════════════════════════════════════════════════\n")

    cm = confusion_matrix(y_test, y_pred, labels=['normal', 'spam', 'urgent'])

    print("📊 Confusion Matrix:")
    print("                    Predicted")
    print("              Normal    Spam    Urgent")
    print(f"    Normal     {cm[0,0]:5d}   {cm[0,1]:5d}    {cm[0,2]:5d}")
    print(f"    Spam       {cm[1,0]:5d}   {cm[1,1]:5d}    {cm[1,2]:5d}")
    print(f"    Urgent     {cm[2,0]:5d}   {cm[2,1]:5d}    {cm[2,2]:5d}")

    print("\n══════════════════════════════════════════════════════════════════════════════════════════\n")

##**4. Test**

**4.1. Training Process:**

In [14]:
import os

csv_path = "/content/drive/MyDrive/Task2/spam.csv"     # SMS Spam Collection from Kaggle (ham / spam)

dataset = DatasetBuilder()
df, stats = dataset.load_data(csv_path)
classifier = MessageClassifier()
results, X_test, y_test, best_model_name = classifier.train(df)
show_detailed_report(results, X_test, y_test, best_model_name)

print("🏁🏁🏁🏁🏁🏁Training Process is Compeletly Done!🏁🏁🏁🏁🏁🏁")

══════════════════════════════════════════════════════════════════════════════════════════

📊 Dataset Information: {'total': 6072, 'urgent': 500, 'normal': 4825, 'spam': 747}
📊 Train: 4,857, Test: 1,215

══════════════════════════════════════════════════════════════════════════════════════════

🎯 Training Models:

▸ Naive_Bayes: 
✓ Accuracy: 93.83% | CV: 93.27% (±0.35%)

▸ Logistic: 
✓ Accuracy: 93.58% | CV: 93.06% (±0.45%)

▸ Random_Forest: 
✓ Accuracy: 89.30% | CV: 88.63% (±0.77%)

▸ Neural_Network: 
✓ Accuracy: 93.17% | CV: 92.63% (±0.26%)

══════════════════════════════════════════════════════════════════════════════════════════

🏆 Best Model: Naive Bayes
   • Accuracy: 93.83%
   • CV Score: 93.27%

══════════════════════════════════════════════════════════════════════════════════════════

📈 classification report (Naive Bayes)

              precision    recall  f1-score   support

      normal      0.933     0.994     0.962       966
        spam      0.959     0.946     0.953    

**4.2. Test some scenatios:**

In [16]:
test_scenarios = {
    "Normal Messages": [
        "Hey, can we meet for coffee tomorrow at 3pm?",
        "Thanks for sending the report. Looks good!",
        "Happy birthday! Hope you have a wonderful day",
        "Dinner tonight? Let me know if you're free",
        "The presentation went really well today"
    ],

    "Spam Messages": [
        "FREE! Win iPhone 15 Pro! Click now: www.scam.com",
        "Congratulations! You won $1,000,000! Text YES to claim prize",
        "Hot singles in your area want to meet you! Join now!",
        "Limited time offer! 90% discount on all products! Buy now!",
        "You have been selected for a free cruise! Call 555-1234"
    ],

    "Urgent Messages": [
        "URGENT: Server is down! Please check immediately!",
        "EMERGENCY: Critical security breach detected in system",
        "ASAP: Boss needs the presentation in 10 minutes",
        "CRITICAL: Production bug affecting all users",
        "ATTENTION: Meeting moved to now, please join"
    ],

    "Challenging Cases": [
        "LOL just kidding! But seriously we need to meet asap 😂",
        "Important discount! Limited time offer! Buy now or never!",
        "Please review the urgent matter we discussed yesterday",
        "OMG you won't believe this amazing deal I found!",
        "Can you PLEASE respond ASAP? It's about the project"
    ]
}

print("\n══════════════════════════════════════════════════════════════════════════════════════════\n")
for category, messages in test_scenarios.items():
    print(f"✅✅✅ {category} ✅✅✅")

    predictions = classifier.predict_batch(messages)

    for idx, row in predictions.iterrows():
        print(f"\n  message: {row['message']}")
        print(f"           prediction: {row['predicted'].upper()} ({row['confidence']})")
        print(f"           probabilities: Normal={row['prob_normal']} | Spam={row['prob_spam']} | Urgent={row['prob_urgent']}")

    print("\n══════════════════════════════════════════════════════════════════════════════════════════\n")



══════════════════════════════════════════════════════════════════════════════════════════

✅✅✅ Normal Messages ✅✅✅

  message: Hey, can we meet for coffee tomorrow at 3pm?
           prediction: NORMAL (99.3%)
           probabilities: Normal=99.3% | Spam=0.4% | Urgent=0.3%

  message: Thanks for sending the report. Looks good!
           prediction: NORMAL (98.6%)
           probabilities: Normal=98.6% | Spam=0.8% | Urgent=0.6%

  message: Happy birthday! Hope you have a wonderful day
           prediction: NORMAL (96.7%)
           probabilities: Normal=96.7% | Spam=0.1% | Urgent=3.3%

  message: Dinner tonight? Let me know if you're free
           prediction: NORMAL (96.4%)
           probabilities: Normal=96.4% | Spam=0.1% | Urgent=3.5%

  message: The presentation went really well today
           prediction: NORMAL (97.8%)
           probabilities: Normal=97.8% | Spam=0.1% | Urgent=2.1%

══════════════════════════════════════════════════════════════════════════════════════════

**4.3. Test custom messages**

In [18]:
def test_custom_message(message):
    predictions = classifier.predict_batch([message])
    row = predictions.iloc[0]

    print(f"\nmessage: {message}")
    print(f"         prediction: {row['predicted'].upper()} ({row['confidence']})")
    print(f"         probabilities: Normal={row['prob_normal']} | Spam={row['prob_spam']} | Urgent={row['prob_urgent']}")



sample_messages = [
    "Can we reschedule our meeting to next week?",
    "FREE iPhone giveaway! Click here now!!!",
    "URGENT: Critical bug in production needs immediate fix",
    "Thanks for your help yesterday, really appreciate it",
    "it's not urgent. sleep.",
    "click here to buy new collection",
    "call me asalp",
    "Server is down! Please check immediately!"
]

for msg in sample_messages:
    test_custom_message(msg)


message: Can we reschedule our meeting to next week?
         prediction: NORMAL (87.4%)
         probabilities: Normal=87.4% | Spam=1.2% | Urgent=11.4%

message: FREE iPhone giveaway! Click here now!!!
         prediction: SPAM (74.2%)
         probabilities: Normal=23.0% | Spam=74.2% | Urgent=2.9%

message: URGENT: Critical bug in production needs immediate fix
         prediction: URGENT (100.0%)
         probabilities: Normal=0.0% | Spam=0.0% | Urgent=100.0%

message: Thanks for your help yesterday, really appreciate it
         prediction: NORMAL (98.7%)
         probabilities: Normal=98.7% | Spam=0.4% | Urgent=0.9%

message: it's not urgent. sleep.
         prediction: NORMAL (100.0%)
         probabilities: Normal=100.0% | Spam=0.0% | Urgent=0.0%

message: click here to buy new collection
         prediction: SPAM (87.5%)
         probabilities: Normal=10.9% | Spam=87.5% | Urgent=1.6%

message: call me asalp
         prediction: URGENT (100.0%)
         probabilities: Normal=0.